In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sopt
from pysimu import ode2numba, ssa

from ipywidgets import *


In [2]:
%matplotlib notebook

## System definition 

In [3]:
sys = { 't_end':20.0,'Dt':0.01,'solver':'forward-euler', 'decimation':10, 'name':'freq2',
   'models':[{'params':
                   {'S_1':1,'B_1':10.0, 'H_1':5.0, 'Droop_1':0.05, 'D_1':10.0,
                    'T_b_1':10.0, # turbine lag time constant, seconds
                    'T_c_1':2.0,    # turbine lead time constant, seconds
                    'K_pgov_1':10.0, # governor proportional gain, per unit
                    'K_igov_1':2,   # governors integral gain, per unit
                    'K_imw_1':0.0, # load (power) controller gain, per unit 
                    'omega_ref_1':1.0,
                    'S_2':1,'B_2':10.0, 'H_2':5.0, 'Droop_2':0.05, 'D_2':10.0,
                    'T_b_2':10.0, # turbine lag time constant, seconds
                    'T_c_2':2.0,    # turbine lead time constant, seconds
                    'K_pgov_2':10.0, # governor proportional gain, per unit
                    'K_igov_2':2,   # governors integral gain, per unit
                    'K_imw_2':0.001, # load (power) controller gain, per unit 
                    'omega_ref_2':1.0,
                    'Omega_b':2*np.pi*50,'S_b':1.0,
                    'p_load': 0.4,
                    'K_p_agc':0.0,
                    'K_i_agc':0.0001
                   },
              'f':[
                    'dphi_1=Omega_b*(omega_1-omega_coi)',  # machine 1 internal angle, rad/s
                    'domega_1 = (p_m_1 - p_1 - D_1*(omega_1-omega_coi))/(2*H_1)', # machine 1 speed, p.u.
                    'dxi_lc_1 = K_imw_1 * (p_ref_1 - p_1)', # machine 1 load control integrator, p.u.
                    'dxi_w_1 = eps_1',                      # machine 1 speed control integrator, p.u.
                    'dx_m_1 = -x_m_1/T_b_1 + valv_1',
                    'dphi_2=Omega_b*(omega_2-omega_coi)',  # machine 1 internal angle, rad/s
                    'domega_2 = (p_m_2 - p_2 - D_2*(omega_2-omega_coi))/(2*H_2)', # machine 1 speed, p.u.
                    'dxi_lc_2 = K_imw_2 * (p_ref_2 - p_2)', # machine 1 load control integrator, p.u.
                    'dxi_w_2 = eps_2',                      # machine 1 speed control integrator, p.u.
                    'dx_m_2 = -x_m_2/T_b_2 + valv_2',
                    'dxi_agc = 1.0 - omega_coi'
                   ],
              'g':[  'p_1@ B_1*sin(phi_1 - phi) - p_1*(S_1/S_b)',
                     'eps_1@-eps_1 + omega_ref_1 - omega_1 + xi_lc_1 - Droop_1*p_1',
                     'valv_1@-valv_1 + K_pgov_1*eps_1 + K_igov_1*xi_w_1',
                     'p_m_1@-p_m_1 + (1/T_b_1 - T_c_1/(T_b_1*T_b_1))*x_m_1 + T_c_1/T_b_1*valv_1',
                     'p_2@B_2*sin(phi_2 - phi) - p_2*(S_2/S_b)',
                     'eps_2@-eps_2 + omega_ref_2 - omega_2 + xi_lc_2 - Droop_2*p_2',
                     'valv_2@-valv_2 + K_pgov_2*eps_2 + K_igov_2*xi_w_2',
                     'p_m_2@-p_m_2 + (1/T_b_2 - T_c_2/(T_b_2*T_b_2))*x_m_2 + T_c_2/T_b_2*valv_2',
                     'phi@ (p_1*S_1/S_b + p_2*S_2/S_b) - p_load',
                     'omega_coi@-omega_coi + (H_1*omega_1 + H_2*omega_2)/(H_1 + H_2)',
                     'p_ref@-p_ref + K_p_agc * (1.0 - omega_coi) + K_i_agc * (xi_agc) ',
                     'p_ref_1@-p_ref_1 + S_1/S_b*p_ref',
                     'p_ref_2@-p_ref_2 + S_2/S_b*p_ref',
                      ],
              'u':{}, 
              'u_ini':{},
              'y_ini':['p_1','eps_1','valv_1','p_m_1','p_2','eps_2','valv_2','p_m_2','phi','omega_coi','p_ref',
                      'p_ref_1','p_ref_2'],
              'h':[
                   'omega_1', 'omega_2'
                   ]}
              ],
    'perturbations':[{'type':'step','time':1.0,'var':'p_load','final':1.0} ]
    }

# B_1, B_2, etc in the system base
# p_load in system base
# H_1, H_2, p_m_1, p_m_2, K_f_1, K_f_2 in machine system base 
x,f = ode2numba.system(sys)  

p_1
eps_1
valv_1
p_m_1
p_2
eps_2
valv_2
p_m_2
phi
omega_coi
p_ref
p_ref_1
p_ref_2
 B_1*sin(phi_1 - phi) - p_1*(S_1/S_b), -eps_1 + omega_ref_1 - omega_1 + xi_lc_1 - Droop_1*p_1, -valv_1 + K_pgov_1*eps_1 + K_igov_1*xi_w_1, -p_m_1 + (1/T_b_1 - T_c_1/(T_b_1*T_b_1))*x_m_1 + T_c_1/T_b_1*valv_1, B_2*sin(phi_2 - phi) - p_2*(S_2/S_b), -eps_2 + omega_ref_2 - omega_2 + xi_lc_2 - Droop_2*p_2, -valv_2 + K_pgov_2*eps_2 + K_igov_2*xi_w_2, -p_m_2 + (1/T_b_2 - T_c_2/(T_b_2*T_b_2))*x_m_2 + T_c_2/T_b_2*valv_2,  (p_1*S_1/S_b + p_2*S_2/S_b) - p_load, -omega_coi + (H_1*omega_1 + H_2*omega_2)/(H_1 + H_2), -p_ref + K_p_agc * (1.0 - omega_coi) + K_i_agc * (xi_agc) , -p_ref_1 + S_1/S_b*p_ref, -p_ref_2 + S_2/S_b*p_ref, 
Matrix([[p_1], [eps_1], [valv_1], [p_m_1], [p_2], [eps_2], [valv_2], [p_m_2], [phi], [omega_coi], [p_ref], [p_ref_1], [p_ref_2]])
N_y 13
 B_1*sym.sin(phi_1 - phi) - p_1*(S_1/S_b), -eps_1 + omega_ref_1 - omega_1 + xi_lc_1 - Droop_1*p_1, -valv_1 + K_pgov_1*eps_1 + K_igov_1*xi_w_1, -p_m_1 + (1/T_b_1